<a href="https://colab.research.google.com/github/oxygen0605/ImageClassification/blob/master/deep_cnn_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratory環境の初期設定

## Google Driveにマウントしてマシンスペックを出力

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

!nvidia-smi > '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /proc/driver/nvidia/gpus/0000:00:04.0/information >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /etc/issue >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /proc/cpuinfo >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls -al

total 20
drwxr-xr-x 1 root root 4096 Aug 19 15:51 .
drwxr-xr-x 1 root root 4096 Aug 19 15:47 ..
drwxr-xr-x 1 root root 4096 Aug 13 16:04 .config
drwx------ 3 root root 4096 Aug 19 15:51 drive
drwxr-xr-x 1 root root 4096 Aug  2 16:06 sample_data


# Deep CNN (CIFAR-10)

## モデルの生成

In [0]:
from keras.models import Sequential, Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Dense, BatchNormalization
from keras.layers import Input
from keras.layers.core import Activation, Flatten
from keras.applications.vgg16 import VGG16
from keras import regularizers

def deep_cnn(input_shape, num_classes):
    inputs = Input(shape = input_shape)
    
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(inputs)
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(512,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(512,(3,3),padding = "SAME",activation= "relu")(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(1024,activation = "relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024,activation = "relu")(x)
    x = Dropout(0.5)(x)
    y  = Dense(num_classes, activation = "softmax")(x)

    return Model(input = inputs, output = y)

## CIFAR10 データセットの用意

In [0]:
import keras
from keras.datasets import cifar10

class CIFAR10Dataset():
	def __init__(self):
		self.image_shape = (32, 32, 3)
		self.num_classes = 10
		
	def preprocess(self, data, label_data=False):
		if label_data:
			# conver class number to one-hot vector
			data = keras.utils.to_categorical(data, self.num_classes)
		
		else:
			data = data.astype("float32")
			data /= 255 #convert the value to 0 ~ 1 scale
			shape = (data.shape[0],) + self.image_shape
			data = data.reshape(shape)
			
		return data
	
	def get_batch(self):
		# x: data, y: lebel
		(x_train, y_train), (x_test, y_test) = cifar10.load_data()
		
		x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
		y_train, y_test = [self.preprocess(d, label_data=True) for d in
					 [y_train, y_test]]
		
		return x_train, y_train, x_test, y_test

## TensorBoard用のログファイル生成関数

In [0]:
from __future__ import absolute_import
from __future__ import unicode_literals
from time import gmtime, strftime
from keras.callbacks import TensorBoard
import os


def make_tensorboard(set_dir_name=''):
    tictoc = strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())
    directory_name = tictoc
    log_dir = set_dir_name + '_' + directory_name
    os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, )
    return tensorboard

## ImageDataGeneratorクラスの拡張
random crop
mix up
cutout
を実装する。

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

class ImageDataGeneratorEX(ImageDataGenerator):
  def __init__(self,
               featurewise_center = False,
               samplewise_center = False, 
               featurewise_std_normalization = False,
               samplewise_std_normalization = False, 
               zca_whitening = False, 
               zca_epsilon = 1e-06, 
               rotation_range = 0.0, 
               width_shift_range = 0.0, 
               height_shift_range = 0.0,
               brightness_range = None,
               shear_range = 0.0,
               zoom_range = 0.0, 
               channel_shift_range = 0.0,
               fill_mode = 'nearest',
               cval = 0.0,
               horizontal_flip = False, 
               vertical_flip = False,
               rescale = None,
               preprocessing_function = None,
               data_format = None,
               validation_split = 0.0, 
               random_crop = None,
               mix_up_alpha = 0.0,
               cutout_mask_size = 0
              ):
    
    # 親クラスのコンストラクタ
    super().__init__(featurewise_center, samplewise_center, featurewise_std_normalization, samplewise_std_normalization, zca_whitening, zca_epsilon, rotation_range, width_shift_range, height_shift_range, brightness_range, shear_range, zoom_range, channel_shift_range, fill_mode, cval, horizontal_flip, vertical_flip, rescale, preprocessing_function, data_format, validation_split)
    # 拡張処理のパラメーター
    # Mix-up
    assert mix_up_alpha >= 0.0
    self.mix_up_alpha = mix_up_alpha
    # Random Crop
    assert random_crop == None or len(random_crop) == 2
    self.random_crop_size = random_crop
    
    # ランダムクロップ
    # 参考 https://jkjung-avt.github.io/keras-image-cropping/
    def random_crop(self, original_img):
        # Note: image_data_format is 'channel_last'
        assert original_img.shape[2] == 3
        if original_img.shape[0] < self.random_crop_size[0] or original_img.shape[1] < self.random_crop_size[1]:
            raise ValueError(f"Invalid random_crop_size : original = {original_img.shape}, crop_size = {self.random_crop_size}")

        height, width = original_img.shape[0], original_img.shape[1]
        dy, dx = self.random_crop_size
        x = np.random.randint(0, width - dx + 1)
        y = np.random.randint(0, height - dy + 1)
        return original_img[y:(y+dy), x:(x+dx), :]

    # Mix-up
    # 参考 https://qiita.com/yu4u/items/70aa007346ec73b7ff05
    def mix_up(self, X1, y1, X2, y2):
        assert X1.shape[0] == y1.shape[0] == X2.shape[0] == y2.shape[0]
        batch_size = X1.shape[0]
        l = np.random.beta(self.mix_up_alpha, self.mix_up_alpha, batch_size)
        X_l = l.reshape(batch_size, 1, 1, 1)
        y_l = l.reshape(batch_size, 1)
        X = X1 * X_l + X2 * (1-X_l)
        y = y1 * y_l + y2 * (1-y_l)
        return X, y
    
    def cutout(self, x, y, seed=None):
        return np.array(list(map(self._cutout, x, seed))), y

    def _cutout(self, image_origin, seed=None):
        # 最後に使うfill()は元の画像を書き換えるので、コピーしておく
        image = np.copy(image_origin)
        mask_value = image.mean()
        
        # 乱数固定(flowでseed固定したら必要ないかも)
        if seed:
          np.random.seed(seed)
          
        h, w, _ = image.shape
        # マスクをかける場所のtop, leftをランダムに決める
        # はみ出すことを許すので、0以上ではなく負の値もとる(最大mask_size // 2はみ出す)
        top = np.random.randint(0 - self.cutout_mask_size // 2, h - self.cutout_mask_size)
        left = np.random.randint(0 - self.cutout_mask_size // 2, w - self.cutout_mask_size)
        bottom = top + self.cutout_mask_size
        right = left + self.cutout_mask_size

        # はみ出した場合の処理
        if top < 0:
            top = 0
        if left < 0:
            left = 0

        # マスク部分の画素値を平均値で埋める
        image[top:bottom, left:right, :].fill(mask_value)
        return image

    def flow(self, *args, **kwargs):
        batches = super().flow(*args, **kwargs)

        # 拡張処理
        while True:
            batch_x, batch_y = next(batches)

            if self.cutout_mask_size > 0:
                result = self.cutout(batch_x, batch_y)
                batch_x, batch_y = result                        

            yield (batch_x, batch_y)
            
ex_gen = ImageDataGeneratorEX(rotation_range=10, horizontal_flip=True, zoom_range=0.1, cutout_mask_size=16)

## Training, Evaluation用クラスの定義

In [0]:
import os
from keras.models import load_model
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import RMSprop, Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

from __future__ import absolute_import
from __future__ import unicode_literals
from time import gmtime, strftime
import os


def make_tensorboard(set_dir_name=''):
    tictoc = strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())
    directory_name = tictoc
    log_dir = set_dir_name + '_' + directory_name
    os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, )
    return tensorboard

class Trainer():
	
	def __init__(self, model, loss, optimizer, logdir = './'):
		self._target = model
		self._target.compile(
				loss=loss, optimizer=optimizer, metrics=["accuracy"]
				)
		self.verbose = 1 # visualize progress bar: 0(OFF), 1(On), 2(On:each data) 
		#self.log_dir = os.path.join(os.path.dirname(__file__), logdir)
		self.log_dir = os.path.join(logdir)
		self.model_file_name = "model_file.hdf5"
	
	def train_for_tuning_test_data(self, 
            x_train, y_train, x_test, y_test, batch_size, epochs, lr_scheduler):
		datagen = ImageDataGenerator(
			featurewise_center=False,            # set input mean to 0 over the dataset
            samplewise_center=False,             # set each sample mean to 0
            featurewise_std_normalization=False, # divide inputs by std
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,                 # apply ZCA whitening
            rotation_range=20,                   # randomly rotate images in the range (0~180)
            width_shift_range=0.2,               # randomly shift images horizontally
            height_shift_range=0.2,              # randomly shift images vertically
            zoom_range = 0.2,
            channel_shift_range = 0.2,
            horizontal_flip=True,                # randomly flip images
            vertical_flip=False                  # randomly flip images
		)
        
    # training (validation dataはデータ拡張はしない)
		model_path = os.path.join(self.log_dir, self.model_file_name)
		self._target.fit_generator(
            generator        = datagen.flow(x_train,y_train, batch_size),
            steps_per_epoch  = x_train.shape[0] // batch_size,
            epochs           = epochs,
            validation_data  = ImageDataGenerator().flow(x_test,y_test, batch_size),
			      validation_steps = x_test.shape[0] // batch_size,
            callbacks=[
                LearningRateScheduler(lr_scheduler),
                make_tensorboard(set_dir_name=self.log_dir),
                ModelCheckpoint(model_path, save_best_only=True,monitor='val_acc',mode='max')
            ],
            verbose = self.verbose,
            workers = 4
        )
		

class Evaluator():
    
    def __init__(self, result_file_path="./prediction_result.csv"):
        self.result_file_path="./prediction_result.csv"
        
    def simple_evaluate(self, model, x_test, label):
        print("start evaluation...")
        score = model.evaluate(x_test, y_test, verbose=1)
        print("Test loss:", score[0])
        print("Test accuracy:", score[1])
        return score
    
    def tta_evaluate(self, model, x_test, label, batch_size = 2500, tta_epochs = 2):
        print("batch size (TTA): "+str(batch_size))
        print("epochs (TTA): "+str(tta_epochs))
        tta = TTA()
        tta_pred = tta.predict(model, x_test, batch_size, epochs = tta_epochs)
        print("Test accuracy(TTA): ",end = "")
        print( accuracy_score( np.argmax(tta_pred,axis = 1) , np.argmax(label,axis = 1)))
        return tta_pred

## 学習率減衰

In [0]:
def learning_rate_schedule_for_Adam(epoch):
	lr = 0.001
	if(epoch >= 100): lr = 0.0002 
	if(epoch >= 200): lr = 0.0001
	return lr

## 実行

In [9]:
from keras.optimizers import Adam
from keras.models import load_model


# create dataset
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()

# create model
model = deep_cnn(dataset.image_shape, dataset.num_classes)

save_dir='/content/drive/My Drive/Colab Notebooks/Logs/deep_cnn_2/'

# train the model
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam(), logdir=save_dir)
#trainer.train_for_tuning_test_data(
#            x_train, y_train, x_test, y_test, batch_size=500, epochs=250, 
#            lr_scheduler=learning_rate_schedule_for_Adam)


# bestなモデルをロードする
#model = load_model(save_dir+trainer.model_file_name)
model = load_model(save_dir+"cnn_best_acc_model.hdf5")

# show result
evaluator = Evaluator()
score = evaluator.simple_evaluate(model, x_test, y_test)

170500096/170498071 [==============================] - 2s 0us/step


W0819 15:52:01.857453 140189237729152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 15:52:01.891460 140189237729152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 15:52:01.899355 140189237729152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 15:52:01.948158 140189237729152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0819 15:52:01.949096 1401892377

start evaluation...
10000/10000 [==============================] - 7s 661us/step
Test loss: 0.2690602608136833
Test accuracy: 0.934


##  Test Time Augmentation（TTA）を用いた推論 

In [14]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

class TTA:
    
    #test_time_augmentation
    #batch_sizeは，test_sizeの約数!!!
    def predict(self, model, x_test, batch_size ,epochs = 10):
        
        # Augmentation用generatorによるデータセットの作成
        data_flow = self.generator(x_test, batch_size)
        
        test_size = x_test.shape[0]
        pred = np.zeros(shape = (test_size,10), dtype = float)
        
        step_per_epoch = test_size //batch_size
        for epoch in range(epochs):
            print( 'epoch: ' + str(epoch+1)+'/'+str(epochs))
            for step in range(step_per_epoch):
                #print( 'step: ' + str(step+1)+'/'+str(step_per_epoch))
                sta = batch_size * step
                end = sta + batch_size
                tmp_x = data_flow.__next__()
                pred[sta:end] += model.predict(tmp_x)        
        return pred / epochs
    
    
    def generator(self, x_test,batch_size):
        return ImageDataGenerator(
                    rotation_range = 10,
                    horizontal_flip = True,
                    height_shift_range = 0.1,
                    width_shift_range = 0.1,
                    zoom_range = 0.1,
                    channel_shift_range = 0.1
                ).flow(x_test,batch_size = batch_size,shuffle = False, seed=756) #756 9447

      
# show result
evaluator = Evaluator()
score = evaluator.tta_evaluate(model, x_test, y_test, batch_size = 500, tta_epochs = 50)

batch size (TTA): 500
epochs (TTA): 100
epoch: 1/100
epoch: 2/100
epoch: 3/100
epoch: 4/100
epoch: 5/100
epoch: 6/100
epoch: 7/100
epoch: 8/100
epoch: 9/100
epoch: 10/100
epoch: 11/100
epoch: 12/100
epoch: 13/100
epoch: 14/100
epoch: 15/100
epoch: 16/100
epoch: 17/100
epoch: 18/100
epoch: 19/100
epoch: 20/100
epoch: 21/100
epoch: 22/100
epoch: 23/100
epoch: 24/100
epoch: 25/100
epoch: 26/100
epoch: 27/100
epoch: 28/100
epoch: 29/100
epoch: 30/100
epoch: 31/100
epoch: 32/100
epoch: 33/100
epoch: 34/100
epoch: 35/100
epoch: 36/100
epoch: 37/100
epoch: 38/100
epoch: 39/100
epoch: 40/100
epoch: 41/100
epoch: 42/100
epoch: 43/100
epoch: 44/100
epoch: 45/100
epoch: 46/100
epoch: 47/100
epoch: 48/100
epoch: 49/100
epoch: 50/100
epoch: 51/100
epoch: 52/100
epoch: 53/100
epoch: 54/100
epoch: 55/100
epoch: 56/100
epoch: 57/100
epoch: 58/100
epoch: 59/100
epoch: 60/100
epoch: 61/100
epoch: 62/100
epoch: 63/100
epoch: 64/100
epoch: 65/100
epoch: 66/100
epoch: 67/100
epoch: 68/100
epoch: 69/100
epo